In [ ]:
#import packages
import numpy as np
import pandas as pd
import datetime as dt
import os 

In [ ]:
#import page
import requests as rq
base_url = 'https://minimisj2.bge-hautsdefrance.fr/contactsLDAP_photo_modal.php'
id= '43017140066119446'
page = rq.get(f"{base_url}?id={id}", auth=('xxxx', 'xxxxx'))
list = page.content
list

In [ ]:
#Beautifulsoup
from bs4 import BeautifulSoup
list_bs = BeautifulSoup(list, 'html.parser')
list_bs

In [ ]:
list_name = list_bs.find_all('h1')
list_name = [name.get_text() for name in list_name]
list_name
len(list_name)

In [ ]:
list_position = list_bs.find_all('h2')
list_position = [position.get_text() for position in list_position]
list_position
len(list_position)

In [ ]:
list_agency = list_bs.find_all('h3')
list_agency = [agency.get_text() for key, agency in enumerate(list_agency) if key % 2 == 0]
list_agency

In [ ]:
type(list_agency)
len(list_agency)

In [ ]:
try:
    list(zip(list_name, list_position, list_agency))

except Exception as ex:
    print(f"An unexpected exception occurred: {ex}")
    # Handle other exceptions

In [ ]:
global_list = {'name' : list_name,
               'position' : list_position,
               'agency' : list_agency}
global_list = pd.DataFrame(global_list)
global_list

In [ ]:
global_list.to_csv('people_global_list.csv', index=False)

SELENIUM

In [ ]:
import selenium 
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

#import packages dynamic scraping
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


In [ ]:
#dynamic website
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException


In [ ]:
try:
    service = Service()
    options = webdriver.ChromeOptions
    driver = webdriver.Chrome(service=service, options=option)
except Exception as ex:
    print(f"An unexpected exception occurred: {ex}")
    # Handle other exceptions

finally:
    if 'driver' in locals() and driver is not None:
        driver.quit()